In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import folium

In [2]:
DATA = '../../data/FridgeGeo150.csv'
COORDINATES = ['Latitude', 'Longitude']

In [3]:
# Read data and get stats
df = pd.read_csv(DATA)

### Get stats

In [4]:
print(f'Data Stats:\n\n{df.describe()}\n')
print(f'Data types:\n\n{df.dtypes}')

Data Stats:

          Loc_key      Latitude     Longitude
count  150.000000  1.480000e+02  1.480000e+02
mean    75.493333  6.760894e+04  6.748097e+04
std     43.453863  8.219914e+05  8.220020e+05
min      1.000000 -1.000000e-06 -8.817333e+01
25%     38.250000  4.186767e+01 -8.790688e+01
50%     75.500000  4.188823e+01 -8.767674e+01
75%    112.750000  4.197813e+01 -8.763310e+01
max    150.000000  9.999999e+06  9.999999e+06

Data types:

Loc_key        int64
Latitude     float64
Longitude    float64
dtype: object


**What stands out?**
- Latitude
    - min value: invalid range (approximately zero)
    - max value: invalid range (too large ~$1e7$)
- Longitude
    - max value: invalid range (too large ~$1e7$)
- Loc_key
    - There are duplicates
        - How? Since min and max are $[1, 150]$, count is equal to $150$, and the column data type are integers then we would assume it to be uniformly distributed. However, the quartile values are incorrect. At $50\%$ quartile we should have $150/2 = 75$ instead of $75.5$.

### Nulls
In the cell below, if an element in the data has a null value then the row index be listed

In [5]:
# Find nulls (if any)
nulls = df[df.isnull().any(axis=1)]
print(f'Number of nulls: {len(nulls)}\nIndex location: {nulls.Loc_key.values}')

Number of nulls: 2
Index location: [35 44]


### Duplicates
In the cell below, we search for complete row duplicates.

In [6]:
# Find duplicate rows
row_dups = df[df.duplicated(keep=False)]
print(f'Number of duplicates: {len(row_dups)}\nEntire row duplicates indices: {row_dups.index.values}')

Number of duplicates: 2
Entire row duplicates indices: [19 20]


**Comment**: Since there is a single duplicate pair, then we can remove one of the rows.

In the cell below, we search for duplicate location keys

In [7]:
key_dups = df[df['Loc_key'].duplicated(keep=False)]
print(f'Number of key duplicates: {len(key_dups)}\nKey dupus indices: {key_dups.index.values}')

Number of key duplicates: 2
Key dupus indices: [19 20]


In the cell below, we search for coordinate duplicates

In [8]:
coordinate_dups = df[df[COORDINATES].duplicated(keep=False)]
print(f'Number of key duplicates: {len(coordinate_dups)}\nKey dupus indices: {coordinate_dups.index.values}')

Number of key duplicates: 85
Key dupus indices: [  0   2   3   4   5   6   8   9  10  11  13  14  15  16  17  18  19  20
  21  22  23  24  26  27  28  29  30  31  32  33  34  35  36  39  41  42
  43  44  45  46  47  48  49  50  51  52  53  54  55  56  57  58  59  60
  61  62  63  64  65  66  73  74  76  78  81  82  83 103 106 109 111 113
 118 124 125 130 136 142 143 144 145 146 147 148 149]


We have shown that there are only two key duplicates (indices: $19$, $20$), so that means $83$ values are repeats. Let's find out if values are pairs, triplets, etc.

In [9]:
# Duplicate coordinates pairing
(coordinate_dups
 .groupby(COORDINATES)
 .count()
 .sort_values(by='Loc_key', ascending=False)
)

# Similar to

# SELECT COUNT(Loc_key), Loc_key
# FROM data
# GROUPBY Latitude, Longitude
# HAVING COUNT(Loc_key) > 1
# ORDER BY 1;

,,Loc_key
Latitude,Longitude,
41.532540,-88.098621,6
41.886991,-87.677014,6
41.978133,-87.719373,5
42.949537,-87.906472,4
41.878963,-87.642108,4
41.964854,-87.658926,4
41.896114,-87.619062,4
41.896216,-87.946179,4
41.785370,-87.597074,3


**Note**: Column Loc_key in this case illustrates the count of keys associated with the lat and long values.

- **Things that come to mind**:
    - Loc_key is probably a key that is only unique to the table (ie won't be used for joins).
    So its possible use is to differentiate between between similiar data or to serve as an index for sorting.
    - There can be multiple fridges per location. Although $6$ seems like a high number for a single location. I would need to ask about this.
    
From now on I'll assume that the data is correct and there are indeed 6 fridges together. For the rest of the exploration I will be plotting the results so having duplicate coordinates (at this scale) will not cause issue.

### Mapping

In order to map without problem let's exclude the nulls and one of the key dups rows.

In [10]:
# Get indices where bad date is found and create index with clean data
err_idx_location = nulls.index.union(key_dups[:1].index)
idx = df.index.difference(err_idx_location)

In [11]:
def build_map(data, index):
    '''
    Instantiate map and add coordinates as labels
    
    Args:
        data: pandas dataframe holding sample data
        index: location where clean data should be

    Returns:
        map_: folium map instance with data points
    ''' 
    
    map_ = folium.Map()
    cleaner_df = data.loc[idx]
    for key in cleaner_df.index:
        coordinates = cleaner_df[COORDINATES].loc[key]
        label = '({:.3f}, {:.3f})'.format(*coordinates.values)
        folium.Marker(coordinates, popup=label).add_to(map_)
    
    return map_

In [12]:
# Display Map
first_map = build_map(df, idx)
first_map

Forgot to remove invalid coordinate unique first described in **Get Stats** section. If we exclude these points, then by examinating the following locations are highlighted:
- Greater Chicago Metro Area
- Milwaukee

Now that we know the areas where fridges reside we can place a filter

In [13]:
filter_string = '(44 > Latitude > 41) & (-87 > Longitude > -89)'
df.query(filter_string).index

Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
            ...
            140, 141, 142, 143, 144, 145, 146, 147, 148, 149],
           dtype='int64', length=146)

In [14]:
filter_string = '(44 > Latitude > 41) & (-87 > Longitude > -89)'
good_data = df.query(filter_string).index
new_idx = (df
           .index
           .intersection(good_data)
           .difference(err_idx_location)
          )
print(f'All the bad data out! Good data indices:\n\n{new_idx}')


All the bad data out! Good data indices:

Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
            ...
            140, 141, 142, 143, 144, 145, 146, 147, 148, 149],
           dtype='int64', length=145)


### Putting it all together

In [15]:
def remove_nulls(df):
    '''Return index locating nulls'''
    nulls = df[df.isnull().any(axis=1)].index
    return nulls


def drop_duplicates(df):
    '''Return index locating duplicates'''
    row_dups = df[df.duplicated(keep=False)].index
    return row_dups


def remove_out_of_range(df, filter_str=''):
    '''Return index of data that is within range if filter_str is provided'''
    try:
        removed = df.query(filter_str).index
        return removed

    except TypeError:
        raise Exception('Pass a boolean string condition to filter out coordinate data')
    

def build_map_v2(data, index=None, center=None):
    '''
    Instantiate map and add coordinates as labels
    
    Args:
        data: pandas dataframe holding cleaned sample data
        index: pandas index where valid data can be found
        center: list of coordiates that serve as a starting point for map

    Returns:
        map_: folium map instance with marked data points
    '''

    map_ = folium.Map(location=center)
    cleaned = data.loc[index]
    for key in cleaned.index:
        coordinates = cleaned[COORDINATES].loc[key]
        label = '({:.3f}, {:.3f})'.format(*coordinates.values)
        folium.Marker(coordinates, popup=label).add_to(map_)
    
    return map_


def main():
    DATA = '../../data/FridgeGeo150.csv'
    COORDINATES = ['Latitude', 'Longitude']
    FILTER = '(44 > Latitude > 41) & (-87 > Longitude > -89)'
    CHICAGO = [41.8781, -87.6298]
    
    # Read data
    df = pd.read_csv(DATA)
    
    # Get index information on bad and good data
    idx_nulls = remove_nulls(df)
    idx_dups = drop_duplicates(df)
    idx_in_range = remove_out_of_range(df, FILTER)
    
    # Only good data
    idx_good = (df
                .index
                .intersection(idx_in_range)
                .difference(idx_nulls)
                .difference(idx_dups)
                )
    plot_map = build_map_v2(df, index=idx_good, center=CHICAGO)
    return plot_map
     

In [16]:
final_map = main()
final_map